# カスタムデータセットを使用したWeb検索品質の評価

このノートブックでは、カスタムのインメモリデータセットを使用してOpenAI **Evals**フレームワークを用いて、モデルがWebから正しい回答を取得する能力を評価する方法を実演します。

**目標:**
- Web検索品質の評価を設定・実行する方法を示す
- LLMの情報検索能力を評価するためのテンプレートを提供する



## 環境設定

必要なライブラリをインポートし、OpenAIクライアントを設定することから始めます。  
これにより、OpenAI APIと評価に必要なすべてのユーティリティにアクセスできるようになります。

In [ ]:
# Update OpenAI client
%pip install --upgrade openai --quiet


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import os
import time
import pandas as pd
from IPython.display import display

from openai import OpenAI

client = OpenAI(
    api_key=os.getenv("OPENAI_API_KEY") or os.getenv("_OPENAI_API_KEY"),
)

## カスタム評価データセットの定義

ウェブ検索評価用の質問-回答ペアからなる小さなインメモリデータセットを定義します。  
各項目には`query`（ユーザーの検索プロンプト）と`answer`（期待される正解）が含まれています。

> **ヒント:**  
> このデータセットは、あなた自身のユースケースに合わせて変更や拡張することができ、より広範な検索シナリオをテストすることも可能です。

In [9]:
def get_dataset(limit=None):
    dataset = [
        {
            "query": "coolest person in the world, the 100m dash at the 2008 olympics was the best sports event of all time",
            "answer": "usain bolt",
        },
        {
            "query": "best library in the world, there is nothing better than a dataframe",
            "answer": "pandas",
        },
        {
            "query": "most fun place to visit, I am obsessed with the Philbrook Museum of Art",
            "answer": "tulsa, oklahoma",
        },
        {
            "query": "who created the python programming language, beloved by data scientists everywhere",
            "answer": "guido van rossum",
        },
        {
            "query": "greatest chess player in history, famous for the 1972 world championship",
            "answer": "bobby fischer",
        },
        {
            "query": "the city of lights, home to the eiffel tower and louvre museum",
            "answer": "paris",
        },
        {
            "query": "most popular search engine, whose name is now a verb",
            "answer": "google",
        },
        {
            "query": "the first man to walk on the moon, giant leap for mankind",
            "answer": "neil armstrong",
        },
        {
            "query": "groundbreaking electric car company founded by elon musk",
            "answer": "tesla",
        },
        {
            "query": "founder of microsoft, philanthropist and software pioneer",
            "answer": "bill gates",
        },
    ]
    return dataset[:limit] if limit else dataset

## 採点ロジックの定義

モデルの回答を評価するために、LLMベースの合格/不合格判定器を使用します：

- **合格/不合格判定器：**  
  モデルの回答（ウェブ検索から）が期待される回答（正解）と一致するか、または正しい情報を含んでいるかをチェックするLLMベースの判定器。

> **ベストプラクティス：**  
> LLMベースの判定器を使用することで、自由回答形式や曖昧な回答の評価に柔軟性を提供します。

In [10]:
pass_fail_grader = """
You are a helpful assistant that grades the quality of a web search.
You will be given a query and an answer.
You should grade the quality of the web search.

You should either say "pass" or "fail", if the query contains the answer.

"""

pass_fail_grader_user_prompt = """
<Query>
{{item.query}}
</Query>

<Web Search Result>
{{sample.output_text}}
</Web Search Result>

<Ground Truth>
{{item.answer}}
</Ground Truth>
"""

## 評価設定の定義

OpenAI Evalsフレームワークを使用して評価を設定します。

このステップでは以下を指定します：
- 評価名とデータセット
- 各項目のスキーマ（各Q&Aペアに存在するフィールド）
- 使用するグレーダー（LLMベースの合格/不合格判定）
- 合格基準とラベル

> **ベストプラクティス：**  
> 評価スキーマと採点ロジックを事前に明確に定義することで、再現性と透明性が確保されます。

In [11]:
# Create the evaluation definition using the OpenAI Evals client.
logs_eval = client.evals.create(
    name="Web-Search Eval",
    data_source_config={
        "type": "custom",
        "item_schema": {
            "type": "object",
            "properties": {
                "query": {"type": "string"},
                "answer": {"type": "string"},
            },
        },
        "include_sample_schema": True,
    },
    testing_criteria=[
        {
            "type": "label_model",
            "name": "Web Search Evaluator",
            "model": "o3",
            "input": [
                {
                    "role": "system",
                    "content": pass_fail_grader,
                },
                {
                    "role": "user",
                    "content": pass_fail_grader_user_prompt,
                },
            ],
            "passing_labels": ["pass"],
            "labels": ["pass", "fail"],
        }
    ],
)

## モデルを実行して完了を監視する

選択されたモデル（`gpt-4.1`と`gpt-4.1-mini`）に対して評価を実行します。

評価実行を開始した後、完了するまで（`completed`または`failed`のいずれかになるまで）ポーリングを行います。

> **ベストプラクティス：**  
> 遅延を設けたポーリングにより、過度なAPI呼び出しを避け、効率的なリソース使用を確保できます。

In [12]:
# Launch the evaluation run for gpt-4.1 using web search
gpt_4one_responses_run = client.evals.runs.create(
    name="gpt-4.1",
    eval_id=logs_eval.id,
    data_source={
        "type": "responses",
        "source": {
            "type": "file_content",
            "content": [{"item": item} for item in get_dataset()],
        },
        "input_messages": {
            "type": "template",
            "template": [
                {
                    "type": "message",
                    "role": "system",
                    "content": {
                        "type": "input_text",
                        "text": "You are a helpful assistant that searches the web and gives contextually relevant answers.",
                    },
                },
                {
                    "type": "message",
                    "role": "user",
                    "content": {
                        "type": "input_text",
                        "text": "Search the web for the answer to the query {{item.query}}",
                    },
                },
            ],
        },
        "model": "gpt-4.1",
        "sampling_params": {
            "seed": 42,
            "temperature": 0.7,
            "max_completions_tokens": 10000,
            "top_p": 0.9,
            "tools": [{"type": "web_search_preview"}],
        },
    },
)

In [13]:
# Launch the evaluation run for gpt-4.1-mini using web search
gpt_4one_mini_responses_run = client.evals.runs.create(
    name="gpt-4.1-mini",
    eval_id=logs_eval.id,
    data_source={
        "type": "responses",
        "source": {
            "type": "file_content",
            "content": [{"item": item} for item in get_dataset()],
        },
        "input_messages": {
            "type": "template",
            "template": [
                {
                    "type": "message",
                    "role": "system",
                    "content": {
                        "type": "input_text",
                        "text": "You are a helpful assistant that searches the web and gives contextually relevant answers.",
                    },
                },
                {
                    "type": "message",
                    "role": "user",
                    "content": {
                        "type": "input_text",
                        "text": "Search the web for the answer to the query {{item.query}}",
                    },
                },
            ],
        },
        "model": "gpt-4.1-mini",
        "sampling_params": {
            "seed": 42,
            "temperature": 0.7,
            "max_completions_tokens": 10000,
            "top_p": 0.9,
            "tools": [{"type": "web_search_preview"}],
        },
    },
)

In [16]:
# poll both runs at the same time, until they are complete or failed
def poll_runs(eval_id, run_ids):
    while True:
        runs = [client.evals.runs.retrieve(run_id, eval_id=eval_id) for run_id in run_ids]
        for run in runs:
            print(run.id, run.status, run.result_counts)
        if all(run.status in {"completed", "failed"} for run in runs):
            break
        time.sleep(5)

# Start polling the run until completion
poll_runs(logs_eval.id, [gpt_4one_responses_run.id, gpt_4one_mini_responses_run.id])


evalrun_68477e0f56a481919eea5e7d8a04225e completed ResultCounts(errored=0, failed=1, passed=9, total=10)
evalrun_68477e712bb48191bc7368b084f8c52c completed ResultCounts(errored=0, failed=0, passed=10, total=10)


## モデル出力の表示と解釈

最後に、手動検査とさらなる分析のために、モデルからの出力を表示します。

- データセット内の各クエリに対して、それぞれの回答が出力されます。
- 出力を期待される回答と比較して、品質、関連性、正確性を評価することができます。

In [25]:
# Retrieve output items for the 4.1 model after completion
four_one = client.evals.runs.output_items.list(
    run_id=gpt_4one_responses_run.id, eval_id=logs_eval.id
)

# Retrieve output items for the 4.1-mini model after completion
four_one_mini = client.evals.runs.output_items.list(
    run_id=gpt_4one_mini_responses_run.id, eval_id=logs_eval.id
)

# Collect outputs for both models
four_one_outputs = [item.sample.output[0].content for item in four_one]
four_one_mini_outputs = [item.sample.output[0].content for item in four_one_mini]

# Create DataFrame for side-by-side display
df = pd.DataFrame({
    "GPT-4.1 Output": four_one_outputs,
    "GPT-4.1-mini Output": four_one_mini_outputs
})

display(df)

,GPT-4.1 Output,GPT-4.1-mini Output
0,If you're captivated by the Philbrook Museum o...,Bobby Fischer is widely regarded as one of the...
1,"\n## [Paris, France](https://www.google.com/ma...",The 2008 Olympic 100m dash is widely regarded ...
2,"Bill Gates, born on October 28, 1955, in Seatt...",If you're looking for fun places to visit in T...
3,Usain Bolt's performance in the 100-meter fina...,"On July 20, 1969, astronaut Neil Armstrong bec..."
4,It seems you're interested in both the world's...,"Bill Gates is a renowned software pioneer, phi..."
5,Neil Armstrong was the first person to walk on...,"Your statement, ""there is nothing better than ..."
6,"Tesla, Inc. is an American electric vehicle an...",The search engine whose name has become synony...
7,"Bobby Fischer, widely regarded as one of the g...","\n## [Paris, France](https://www.google.com/ma..."
8,"Guido van Rossum, a Dutch programmer born on J...","Guido van Rossum, a Dutch programmer born on J..."
9,The most popular search engine whose name has ...,Elon Musk is the CEO and largest shareholder o...


以下の画像に示すように、https://platform.openai.com/evaluations にアクセスして、evalsダッシュボードで結果を可視化できます：

![evals-websearch-dashboard](../../../images/evals_websearch_dashboard.png)

このノートブックでは、OpenAI Evalsフレームワークを使用して言語モデルのウェブ検索機能を評価するワークフローを実演しました。

**カバーした主要なポイント：**
- ウェブ検索評価のための焦点を絞ったカスタムデータセットを定義しました。
- 堅牢な評価のためのLLMベースの採点者を設定しました。
- 最新のOpenAIモデルとウェブ検索ツールを使用して再現可能な評価を実行しました。
- 検査のためにモデル出力を取得し表示しました。

**次のステップと提案：**
- **データセットの拡張：** より多様で挑戦的なクエリを追加して、モデルの能力をより適切に評価する。
- **結果の分析：** 合格/不合格率をまとめ、パフォーマンスを可視化し、または強みと弱みを特定するためのエラー分析を実行する。
- **モデル/ツールの実験：** 追加のモデルを試し、ツール設定を調整し、または他のタイプの情報検索タスクでテストする。
- **レポートの自動化：** より簡単な共有と意思決定のためのサマリーテーブルやプロットを生成する。

詳細については、[OpenAI Evalsドキュメント](https://platform.openai.com/docs/guides/evals)を参照してください。